# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that I delegate the right tasks to the appropriate team members and ask the necessary questions to get the best insights and alerts for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "Utilize statistical modeling and machine learning to identify trends and predict market movements"}

> Entering new CrewAgentExecutor chain...
I need to start by gathering real-time market data for AAPL to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - 

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US Politics 2025 Election World Weather Climate change Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise Women's health Sleep Healthy aging Hearing Mobility Science Originals The 360 Newsletters Games Life Health Wellness Nutrition Fitness Healthy aging Mental health Sleep Your body Dermatology Children's health Foot health Hair loss Hearing Oral health Sexual health Women's health Conditions Cardiovascular health Digestive health Endocrine system Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Style Accessories Clothing Luggage Shoes Beauty Hair Makeup Skincare Sunscreen Health Dental Fitness Hair loss He

Final Answer: 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US Politics 2025 Election World Weather Climate change Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise Women's health Sleep Healthy aging Hearing Mobility Science Originals The 360 Newsletters Games Life Health Wellness Nutrition Fitness Healthy aging Mental health Sleep Your body Dermatology Children's health Foot health Hair loss Hearing aids Mental health Mobility Nutrition Personal care Sleep Women's health Home and garden Bedding Cleaning Gardening Kitchen Outdoor Pets Tech Accessories Audio Auto Computers Phones Smart home TVs Gift ideas Best Advent calendars 2025 Best Christmas gifts Best White Elephant gifts Best Secret Santa gifts Best gifts for m

I need to develop trading strategies for AAPL based on insights from the Data Analyst and user-defined risk tolerance. I also need to consider trading preferences for Day Trading.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop trading strategies for AAPL based on Data Analyst insights and user-defined risk tolerance, considering Day Trading preferences", "context": "AAPL stock price, market trends, user risk tolerance, and trading preferences"}


> Entering new CrewAgentExecutor chain...
I need to gather insights on AAPL stock price, market trends, user risk tolerance, and trading preferences to develop trading strategies. I should use the tools available to gather relevant information.

Action: Search the internet
Action Input: {"search_query": "AAPL stock price and market trends"} 


Search results: Title: Apple Inc. (AAPL) - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/
Snippet: Apple is poised for significan

I need to ensure that the team analyzes the approved trading strategies for AAPL thoroughly to determine the best execution methods based on current market conditions and optimal pricing.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Analyze approved trading strategies for AAPL and determine the best execution methods considering current market conditions and optimal pricing", "context": "After analyzing the insights from the Data Analyst and considering the user's risk tolerance and day trading preferences, the potential trading strategies for AAPL have been identified"}


> Entering new CrewAgentExecutor chain...
I need to gather more information on the approved trading strategies for AAPL to determine the best execution methods.

Action: Search the internet
Action Input: {"search_query": "approved trading strategies for AAPL"} 


Search results: Title: 1 Delicious Options Strategy to Trade Apple Stock at Record Highs
Link: http

I need to gather detailed risk analysis and mitigation recommendations for the proposed trading strategies and execution plans for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Risk Advisor",
"task": "Evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. Provide a detailed analysis of potential risks and suggest mitigation strategies.",
"context": "After analyzing the approved trading strategies for AAPL, the best execution methods considering current market conditions and optimal pricing would involve a combination of various strategies such as option collar, day trading, option strategies, covered calls, and effective trading strategies like day trading, swing trading, and position trading for Apple stock."
}


> Entering new CrewAgentExecutor chain...
I need to carefully analyze the proposed trading strategies for AAPL and assess the potential risks associated with each method. It's crucial to provide detailed insi

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

After evaluating the proposed trading strategies and execution plans for AAPL, the following potential risks have been identified along with corresponding mitigation strategies:

1. Option Collar Strategy:
- Potential Risk: Limited upside potential due to the protective nature of the collar strategy.
- Mitigation Strategy: Consider using a combination of options to enhance potential gains while still protecting against downside risk.

2. Day Trading Strategies:
- Potential Risk: High volatility in the market leading to increased risk of losses.
- Mitigation Strategy: Implement strict stop-loss orders and limit orders to minimize potential losses.

3. Option Strategies:
- Potential Risk: Option pricing fluctuations leading to unexpected losses.
- Mitigation Strategy: Conduct thorough analysis of option pricing and implied volatility before executing trades.

4. Covered Calls:
- Potential Risk: Limited profit potential if the stock price surpasses the call option strike price.
- Mitigation Strategy: Regularly monitor the stock price and consider rolling the call options to capture additional gains.

5. Effective Trading Strategies:
- Potential Risk: Market uncertainty impacting the effectiveness of trading strategies.
- Mitigation Strategy: Stay informed about market conditions and adjust trading strategies accordingly to adapt to changing circumstances.

Overall, it is crucial for traders to stay vigilant, continuously monitor market conditions, and be prepared to adjust their strategies to mitigate risks and maximize profitability when executing trades for AAPL.